In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr

/home/pavel/.local/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [6]:
yf.pdr_override()

In [9]:
startyear=2019
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)
now=dt.datetime.now()

df=pdr.get_data_yahoo('SPY',start,now)

[*********************100%***********************]  1 of 1 completed


In [10]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,267.839996,268.809998,267.399994,268.769989,256.217468,86655700
2018-01-03,268.959991,270.640015,268.959991,270.470001,257.838104,90070400
2018-01-04,271.200012,272.160004,270.540009,271.609985,258.924835,80636400
2018-01-05,272.510010,273.559998,271.950012,273.420013,260.650299,83524000
2018-01-08,273.309998,274.100006,272.980011,273.920013,261.126984,57319200
...,...,...,...,...,...,...
2020-06-15,298.019989,308.279999,296.739990,307.049988,305.704712,135782700
2020-06-16,315.480011,315.640015,307.670013,312.959991,311.588806,137627500
2020-06-17,314.070007,314.390015,310.859985,311.660004,310.294525,82954600


In [13]:
emasUsed=[3,5,8,10,12,15,30,35,40,45,50,60]

for x in emasUsed:
    ema=x
    df["Ema_"+str(ema)]=round(df.iloc[:,4].ewm(span=ema,
                                               adjust=False).mean(),2)

In [15]:
df = df.iloc[60:]

In [20]:
pos = 0
num = 0
perc = []

for i in df.index:
    cmin=min(df["Ema_3"][i],df["Ema_5"][i],df["Ema_8"][i],df["Ema_10"][i],df["Ema_12"][i],df["Ema_15"][i],)
    cmax=max(df["Ema_30"][i],df["Ema_35"][i],df["Ema_40"][i],df["Ema_45"][i],df["Ema_50"][i],df["Ema_60"][i],)
    
    close = df["Adj Close"][i]
    
    if(cmin > cmax):
        if(pos==0):
            bp=close
            pos=1
            print("Buing now at "+str(bp))
    
    elif(cmin<cmax):
        if(pos==1):
            pos=0
            sp=close
            print("Selling now at "+str(sp))
            pc=(sp/bp-1)*100
            perc.append(pc)
    if(num==df["Adj Close"].count()-1 and pos==1):
        pos=0
        sp=close
        print("Selling now at "+str(sp))
        pc=(sp/bp-1)*100
        perc.append(pc)

    num+=1

Buing now at 261.2731628417969
Selling now at 260.54266357421875
Buing now at 264.7921447753906
Selling now at 277.9662780761719
Buing now at 262.21112060546875
Selling now at 279.68927001953125
Buing now at 282.5382080078125
Selling now at 286.89288330078125
Buing now at 292.2166442871094
Selling now at 288.82843017578125
Buing now at 294.38360595703125
Selling now at 319.1308288574219
Buing now at 291.1587219238281
Selling now at 308.6400146484375


In [23]:
gains=0
ng=0
losses=0
nl=0
totalR=1

for i in perc:
    if(i>0):
        gains+=i
        ng+=1
    else:
        losses+=i
        nl+=1
    totalR=totalR*((i/100)+1)

totalR=round((totalR-1)*100,2)

if(ng>0):
    avgGain=gains/ng
    maxR=str(max(perc))
else:
    avgGain=0
    maxR="undefined"

if(nl>0):
    avgLoss=losses/nl
    maxL=str(min(perc))
    ratio=str(-avgGain/avgLoss)
else:
    avgLoss=0
    maxL="undefined"
    ratio="inf"

if(ng>0 or nl>0):
    battingAvg=ng/(ng+nl)
else:
    battingAvg=0

In [28]:
print()
print("Results for "+ "SPY" +" going back to "+str(df.index[0])+", Sample size: "+str(ng+nl)+" trades")
print("EMAs used: "+str(emasUsed))
print("Batting Avg: "+ str(battingAvg))
print("Gain/loss ratio: "+ ratio)
print("Average Gain: "+ str(avgGain))
print("Average Loss: "+ str(avgLoss))
print("Max Return: "+ maxR)
print("Max Loss: "+ maxL)
print("Total return over "+str(ng+nl)+ " trades: "+ str(totalR)+"%" )
print()


Results for SPY going back to 2018-03-29 00:00:00, Sample size: 7 trades
EMAs used: [3, 5, 8, 10, 12, 15, 30, 35, 40, 45, 50, 60]
Batting Avg: 0.7142857142857143
Gain/loss ratio: 7.6695480602375365
Average Gain: 5.5185435155889895
Average Loss: -0.7195395963680906
Max Return: 8.40645416375625
Max Loss: -1.159487037295226
Total return over 7 trades: 28.78%

